In [1]:
import psycopg2
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) #show all columns
pd.set_option('max_colwidth',None)

In [2]:
def search_table(s='*', f='admissions', w='', like='', download_as=''):
    temp=s.lower()
    for i in temp:
        if i>"z" or i<"a":
            assert "TypeError"
    del temp
    conn = psycopg2.connect(database="mimic", user="postgres",
                            password="pandora", host="127.0.0.1", port="5432")
    cursor = conn.cursor()
    if (w!='')&(like!=''):
        cursor.execute("select " + s + " from mimiciii." + f +' where '+ w +
                       ' like ' + "'%"+ like +"%'")
    elif (w!='')&(like=='') :
        cursor.execute("select " + s + " from mimiciii." + f +' where '+ w)
    else:
        cursor.execute("select " + s + " from mimiciii." + f)
    table = cursor.fetchall()
    if(len(table)!=0):
        colnames = [desc[0] for desc in cursor.description]
        columns = cursor.fetchall()
        cursor.close()
        conn.close()
        columns=pd.DataFrame(columns)
        table=pd.DataFrame(table)
        table.columns=colnames
    if(download_as!=''):
        table.to_csv('download data/'+download_as+'.csv', index=False)
    return table

In [ ]:
res=search_table(s='*', f='SAPSII')
res

In [ ]:
def searchDiseaseID(disease_name=''):
    result=search_table(s='*' ,f='d_icd_diagnoses', w='long_title', like=disease_name , download_as='')
    index_list=[]
    if (len(search_table(s='*' ,f='d_icd_diagnoses', w='long_title', like=disease_name , download_as=''))!=0):
        index_list=np.array(result['icd9_code'])

    return index_list, result
    
index, result=searchDiseaseID('myocardial')


In [ ]:
def searchDisease(index=[], download_as=''):
    res=pd.DataFrame()
    index=list(index)
    count=0
    for i in index:
        count+=1
        if(len(search_table(s='*' ,f='diagnoses_icd', w="icd9_code="+"'"+str(i)+"'", like='' , download_as=''))==0):
            index.remove(str(i))
        
        temp=search_table(s='*' ,f='diagnoses_icd', w="icd9_code="+"'"+str(i)+"'", like='' , download_as='')
        res=res.append(temp, ignore_index=True)
    print('search', count, 'of',len(index))
    if download_as!='':
        res.to_csv('download data/'+download_as+'.csv', index=False)
    return res
        
